In [217]:
# import Panda dependency
import pandas as pd

In [218]:
# filepath as strings, read as DFs
sch_file = "resources\schools_complete.csv"
stu_file = "resources\students_complete.csv"
sch_df = pd.read_csv(sch_file)
stu_df = pd.read_csv(stu_file)

In [222]:
# clean student names using list, looping through, and using replace if string is found in column
presuffixes = ["Dr. ", "Mr. ","Ms. ", "Mrs. ", "Miss ", " MD", " DDS", " DVM", " PhD"]

for psfix in presuffixes:
    stu_df["student_name"] = stu_df["student_name"].str.replace(psfix, "")

C:\Users\nabil\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [223]:
# merge both dfs for complete data set get basic metrics
all_data = pd.merge(stu_df, sch_df, on=["school_name", "school_name"])

student_count = all_data["Student ID"].count()
school_count = len(all_data["school_name"].unique())
total_budget = sch_df["budget"].sum()
mean_read = all_data["reading_score"].mean()
mean_math = all_data["math_score"].mean()

#pass or fail into new DFs, do counts, and percentages
pass_math = all_data[all_data["math_score"] > 69]
pass_read = all_data[all_data["reading_score"] > 69]
pass_both = all_data[(all_data["math_score"] > 69) & (all_data["reading_score"] > 69)]

math_pass_count = pass_math["student_name"].count()
read_pass_count = pass_read["student_name"].count()
pass_both_count = pass_both["student_name"].count()

math_perc = math_pass_count / float(student_count) * 100
read_perc = read_pass_count / float(student_count) * 100
both_perc = pass_both_count / float(student_count) * 100

In [225]:
#create summary df
dist_sum_df = pd.DataFrame(
    [{"Total Schools" : school_count,
    "Total Students" : student_count,
    "Total Budget" : total_budget,
    "Ave Math Score" : mean_math,
    "Ave Read Score" : mean_read,
    "% Pass Math" : math_perc,
    "% Pass Read" : read_perc,
    "% Overall Pass" : both_perc}])

#format each column appropriately
dist_sum_df["Total Students"] = dist_sum_df["Total Students"].map("{:,}".format)
dist_sum_df["Ave Math Score"] = dist_sum_df["Ave Math Score"].map("{:.1f}".format)
dist_sum_df["Ave Read Score"] = dist_sum_df["Ave Read Score"].map("{:.1f}".format)
dist_sum_df["% Pass Math"] = dist_sum_df["% Pass Math"].map("{:.0f}".format)
dist_sum_df["% Pass Read"] = dist_sum_df["% Pass Read"].map("{:.0f}".format)
dist_sum_df["% Overall Pass"] = dist_sum_df["% Overall Pass"].map("{:.0f}".format)
dist_sum_df["Total Budget"] = dist_sum_df["Total Budget"].map("${:,.2f}".format)

#reorder columns appropriately
col_order = ["Total Schools","Total Students", "Total Budget", "Ave Math Score", 
             "Ave Read Score", "% Pass Math", "% Pass Read", "% Overall Pass"]
dist_sum_df = dist_sum_df[col_order]

dist_sum_df

,Total Schools,Total Students,Total Budget,Ave Math Score,Ave Read Score,% Pass Math,% Pass Read,% Overall Pass
0,15,"39,170","$24,649,428.00",79.0,81.9,75,86,65
